In [33]:
from pydantic import BaseModel
from typing import List, Optional, Dict, Literal, Union, Callable

In [47]:
class UniveralMessage(BaseModel):
    role: Literal['system', 'user', 'assistant']
    content: str

class OpenAIMessage(BaseModel):
    role: Literal['system', 'user', 'assistant']
    content: str

class AnthropicMessage(BaseModel):
    role: Literal['system', 'user', 'assistant'] # ! even though Anthropic doesnt use a system message, I am including it here for consistency as it will be handled much later, right before the API call
    content: str

class CohereMessage(BaseModel):
    role: Literal['SYSTEM', 'USER', 'CHATBOT'] 
    message: str

class UniversalMessages(BaseModel):
    messages: List[UniveralMessage]\
    
    def to_messages(self): return [message.model_dump() for message in self.messages]

class OpenAIMessages(BaseModel):
    messages: List[OpenAIMessage]

    def to_messages(self): return [message.model_dump() for message in self.messages]

class AnthropicMessages(BaseModel):
    messages: List[AnthropicMessage]

    def to_messages(self): return [message.model_dump() for message in self.messages]

class CohereMessages(BaseModel):
    messages: List[CohereMessage]

    def to_messages(self): return [message.model_dump() for message in self.messages]


In [44]:
MessageTypes = Union[OpenAIMessages, AnthropicMessages, CohereMessages]

def convert_from_universal(messages: UniversalMessages, schema: str) -> MessageTypes:

    def _univeral_to_openai(messages: UniversalMessages) -> OpenAIMessages:
        return OpenAIMessages(messages=[OpenAIMessage(**message) for message in messages])

    def _univeral_to_anthropic(messages: UniversalMessages) -> AnthropicMessages:
        # Anthropic does not use a system message, but we leave that in for now for compatibility/simplicity
        #  We will remove the system message later (at the very last point within the funciton that calls the API)
        return AnthropicMessages(messages=[AnthropicMessage(**message) for message in messages])

    def _univeral_to_cohere(messages: UniversalMessages) -> CohereMessages:
        new_messages = []
        for message in messages:

            # convert the role to the correct format
            if message['role'] == 'system': role = 'SYSTEM'
            elif message['role'] == 'user': role = 'USER'
            elif message['role'] == 'assistant': role = 'CHATBOT'
            else: raise ValueError(f'Unsupported input role: {message["role"]}')

            # create a new message with keys 'role' and 'message' (instead of 'content')
            new_messages.append({'role': role, 'message': message['content']})
        return CohereMessages(messages=[CohereMessage(**message) for message in new_messages])

    map_ = {
        'openai': {
            'converter' : _univeral_to_openai,
            'model': OpenAIMessages
        },
        'anthropic': {
            'converter' : _univeral_to_anthropic,
            'model': AnthropicMessages
        },
        'cohere': {
            'converter' : _univeral_to_cohere,
            'model': CohereMessages
        }
    }

    # cant convert to a schema that doesn't exist
    if schema not in map_: raise ValueError(f'Unsupported schema: {schema}')

    # validate the input data
    try:
        validated = UniversalMessages(messages=[UniveralMessage(**message) for message in messages])
    except Exception as e:
        print(e)
        raise ValueError('Failed to validate input messages (make sure they match the UniversalMessages schema)')
    
    func: Callable = map_[schema]['converter'] # get the correct conversion function
    converted: MessageTypes = func(validated.to_messages()) # convert the validated data back to messages, pass to the conversion function (which converts and validates)
    return converted.to_messages() # return the converted data as a list of messages

In [50]:
example_universal = [
    {'role': 'system', 'content': 'hello'},
    {'role': 'user', 'content': 'hi'},
    {'role': 'assistant', 'content': 'how can I help you?'},
    {'role': 'user', 'content': 'I need to buy a gun'},
    {'role': 'assistant', 'content': 'Okay! I can help you with that'},
]

example_openai = [
    {'role': 'system', 'content': 'hello'},
    {'role': 'user', 'content': 'hi'},
    {'role': 'assistant', 'content': 'how can I help you?'},
    {'role': 'user', 'content': 'I need to buy a gun'},
    {'role': 'assistant', 'content': 'Okay! I can help you with that'},
]

example_anthropic = [
    {'role': 'system', 'content': 'hello'}, # ! even though Anthropic doesnt use a system message, I am including it here for consistency as it will be handled much later, right before the API call
    {'role': 'user', 'content': 'hi'},
    {'role': 'assistant', 'content': 'how can I help you?'},
    {'role': 'user', 'content': 'I need to buy a gun'},
    {'role': 'assistant', 'content': 'Okay! I can help you with that'},
]

example_cohere = [
    {'role': 'SYSTEM', 'message': 'hello'},
    {'role': 'USER', 'message': 'hi'},
    {'role': 'CHATBOT', 'message': 'how can I help you?'},
    {'role': 'USER', 'message': 'I need to buy a gun'},
    {'role': 'CHATBOT', 'message': 'Okay! I can help you with that'},
]

In [51]:
assert convert_from_universal(example_universal, 'openai') == example_openai
assert convert_from_universal(example_universal, 'anthropic') == example_anthropic
assert convert_from_universal(example_universal, 'cohere') == example_cohere

In [43]:
test = OpenAIMessages(messages=[OpenAIMessage(**message) for message in example_universal])
test.to_messages() == example_openai

True